In [1]:
import pandas as pd

In [4]:
# Import
import string
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize.treebank import TreebankWordDetokenizer
import csv
import re
from tqdm import tqdm
tqdm.pandas()
nltk.download('stopwords')
nltk.download('punkt')
stopwords_id = nltk.corpus.stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\satri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\satri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# Read data
df = pd.read_excel('Data/Tangkuban Parahu.xlsx')

# Tanggal
df = df[df['Date'] >= '2023-01-01'].reset_index(drop=True)

# Dropna
df = df.dropna(subset=['Review']).reset_index(drop=True)

# Fungsi translated
def get_trans(review, translated):
    if str(translated) == 'nan':
        teks = review
    else:
        teks = translated
    return teks

df['Review'] = df.apply(lambda x: get_trans(x['Review'], x['Review Translated']), axis=1)

# Mengambil kolom2 yang diperlukan saja
df = df[['Review']]

df

,Review
0,"Wah ini tempat bersejarah yang ada di bandung,..."
1,Butuh effort buat ksini macet-macetan tapi sam...
2,Seru
3,Petugas dokumntasi foto sangat mengganggu
4,Bagus kali
...,...
1365,Bersih pantainya bagus pengaman nya diluar pan...
1366,"Pemandanganya bagus, ditambah dengan udaranya ..."
1367,Bagus
1368,Plus:\n1. Kondisi alam nya masih terjaga.\n\nM...


In [6]:
# Preprocessing

# Membuat fungsi cleaning_text

# create stemmer (menjadikan setiap kata menjadi kata dasar)
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# remove emoji 
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

def cleaning_text(teks):
    # Membuat tulisan non kapital
    teks = teks.lower()

    # Menghapus newline, contoh: “\n”.
    teks = teks.replace('\n','')
    teks = teks.replace('yg','')
    teks = teks.replace('-',' ')
    teks = teks.replace('_',' ')


    # Menghapus alamat URL yang terdapat pada teks
    teks = re.sub(r'http\S+', '', teks)

    # Menghapus mention yang ada pada teks -> @blabla
    teks = re.sub(r'@[A-Za-z0-9_]+', '', teks)

    # Menghapus hashtag yang ada pada teks -> #blabla
    teks = re.sub(r'#[A-Za-z0-9_]+', '', teks)

    # Menghapus angka yang ada pada teks
    teks = re.sub(r'\b\d+\b', '', teks)
    teks = re.sub(r'[0-9]', '', teks)

    # Menghapus tanda baca yang terdapat pada teks
    teks = teks.translate(str.maketrans('','',string.punctuation))

    # Menghilangkan stopwords atau kata-kata yang kurang bernilai
    text_id_filtered = []
    token_teks = word_tokenize(teks)
    for w in token_teks:
        if w not in stopwords_id:
            w = stemmer.stem(w)
            text_id_filtered.append(w)
    teks = ' '.join(text_id_filtered)

    teks = emoji_pattern.sub(r'', teks)

    return teks

# Menjalan fungsi untuk kolom baru bernama tweet_clean
df['Review_clean'] = df['Review'].progress_apply(cleaning_text)

# Mengatur ulang urutan kolom
df = df[['Review', 'Review_clean']]

df

100%|██████████| 1370/1370 [04:40<00:00,  4.89it/s]


,Review,Review_clean
0,"Wah ini tempat bersejarah yang ada di bandung,...",sejarah bandung lokasi lembang yakali wisata l...
1,Butuh effort buat ksini macet-macetan tapi sam...,butuh effort ksini macet macet bayar lunas set...
2,Seru,seru
3,Petugas dokumntasi foto sangat mengganggu,tugas dokumntasi foto ganggu
4,Bagus kali,bagus kali
...,...,...
1365,Bersih pantainya bagus pengaman nya diluar pan...,bersih pantai bagus kam nya luar pantai
1366,"Pemandanganya bagus, ditambah dengan udaranya ...",pemandanganya bagus tambah udara sejuk
1367,Bagus,bagus
1368,Plus:\n1. Kondisi alam nya masih terjaga.\n\nM...,plus kondisi alam nya terjagaminus tiket masuk...


In [40]:
df.to_excel('Data/Tangkuban Parahu Text Clean.xlsx', index=False)